In [7]:
model_name = 'ReduceAFP' # 'change name here
dim = 256

In [8]:
from minisom import MiniSom
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import rdkit
from rdkit import Chem, DataStructs
from rdkit.Chem import Draw, AllChem
%matplotlib inline

# data process

In [9]:
path = '/home/kongyue/private-kongyue/paper-RGNN/SOM/fingerprints/' + model_name + '_test.csv'
df = pd.read_csv(path)
df['confusion'] = np.NAN
sample_num = df.shape[0]
tp_sel = (df['Serine_threonine-protein_kinase_Aurora-A'] == 1) & (df['pred'] == 1)
tn_sel = (df['Serine_threonine-protein_kinase_Aurora-A'] == 0) & (df['pred'] == 0)
fp_sel = (df['Serine_threonine-protein_kinase_Aurora-A'] == 0) & (df['pred'] == 1)
fn_sel = (df['Serine_threonine-protein_kinase_Aurora-A'] == 1) & (df['pred'] == 0)
df.loc[tp_sel,'confusion'] = 0
df.loc[tn_sel,'confusion'] = 1
df.loc[fp_sel,'confusion'] = 2
df.loc[fn_sel,'confusion'] = 3
df.confusion = df.confusion.astype(int)

In [10]:
fps = df.iloc[:, 5:-1].values
ys = df['Serine_threonine-protein_kinase_Aurora-A'].values
y_preds = df['pred'].values
confusions = df['confusion'].values
label_names = {0:'TP',1:'TN',2:'FP',3:'FN'}
colors = ['C0', 'C1', 'C2', 'C3']

In [12]:
n_neurons = 10
m_neurons = 10
som = MiniSom(n_neurons,m_neurons,dim, sigma=1.5, learning_rate=0.5,
            neighborhood_function='gaussian', random_seed=0)
som.pca_weights_init(fps)
som.train(fps,100000)


/home/kongyue/anaconda3/envs/t/lib/python3.7/site-packages/minisom.py:370: ComplexWarning: Casting complex values to real discards the imaginary part
  self._weights[i, j] = c1*pc[pc_order[0]] + c2*pc[pc_order[1]]


# save model

In [13]:
out_path = '/home/kongyue/private-kongyue/paper-RGNN/SOM/models/' + model_name +'_som.p'
with open(out_path, 'wb') as outfile:
    pickle.dump(som, outfile)